### Load libraries

In [ ]:
from sys import argv
import numpy as np
import pandas as pd
import scipy as sp
from scipy import ndimage
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize
import shapefile as sf
from scipy.interpolate import RegularGridInterpolator
from gnam.model.gridmod3d import gridmod3d as gm
from gnam.model.bbox import bbox as bb
from shapely.geometry import Point, Polygon

### Read in previously computed smoothed model

In [ ]:
#this is a pickled dictionary with 4D ndarray, and 1D meta data arrays
ifilename = './smooth_z10m_nam_model_vp_vs_rho_Q_props.npz'

#Unpickle
data = np.load(ifilename)
props = data['props'] #4D ndarray

#meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('xd:\n',xdata)
print('yd:\n',ydata)
print('zd:\n',zdata)

# Setup Coordinate related vars
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz

nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d:\n',gm3d)

#free up some memory
del props 

In [ ]:
print('gm3d:\n',gm3d)

### Plot a depth slice

In [ ]:
hires_surf = gm3d[0,:,:,200].T
print('hires_surf.shape:',hires_surf.shape)
gxyc = gm3d.getGlobalCoordsPointsXY()
#get min max to normalize surface
hires_vp_min = np.min(hires_surf)
hires_vp_max = np.max(hires_surf)
print('vp_min:',hires_vp_min)
print('vp_max:',hires_vp_max)
hires_norm = Normalize(hires_vp_min,hires_vp_max)

fig, ax = plt.subplots(1,figsize=(8,6))
sc = ax.scatter(gxyc[:,0],gxyc[:,1],s=1,c=hires_surf.flatten(),cmap=plt.cm.jet,norm=hires_norm)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

### Unpickle bounding box

In [ ]:
import pickle

f = open('./model_bbox.pickle', 'rb')
dill_bbox = pickle.load(f)
f.close()
print('bbox after pickle :\n',dill_bbox)

### Plot bounding box on top of surface

In [ ]:

fig, ax = plt.subplots(1,figsize=(8,6))
sc = ax.scatter(gxyc[:,0],gxyc[:,1],s=1,c=hires_surf.flatten(),cmap=plt.cm.jet,norm=hires_norm,zorder=0)
ax.plot(dill_bbox.getCLoop()[:,0],dill_bbox.getCLoop()[:,1],c='black',linestyle='dotted',zorder=4)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

In [ ]:
test_slice = gm3d.slice_volume_by_bbox(dill_bbox)